In [ ]:
import numpy as np

In [ ]:
GRID_SIZE = 3 ** 2
NB_INDIVIDUALS_POPULATION = 50
FITNESS_GOAL = GRID_SIZE * GRID_SIZE * 2

In [ ]:
def generate_individual():
    """
    Initialize a random Sudoku grid where each cell has an equal chance of being a number from 1 to 9.
    So the grid can be invalid. But aftern, we will use a genetic to make the grid valid.

    Returns:
        A numpy.ndarray with dimensions (GRID_SIZE, GRID_SIZE) where each cell contains a value between 1 and GRID_SIZE (inclusive).
    """
    min_value = 1
    max_value = GRID_SIZE + 1
    size = (GRID_SIZE, GRID_SIZE)
    return np.random.randint(min_value, max_value, size=size)


In [ ]:
generate_individual()

In [ ]:
def generate_population():
    """
    Generates a population by creating a list of individuals using the generate_individual function.

    Returns:
        list: A population consisting of multiple individuals.
    """
    return [generate_individual() for x in range(NB_INDIVIDUALS_POPULATION)]

In [ ]:
def compute_fitness(individual):
    """
    Computes the fitness score of an individual.

    Args:
        individual (numpy.ndarray): A numpy.ndarray with dimensions (GRID_SIZE, GRID_SIZE) where each cell contains a value between 1 and GRID_SIZE (inclusive).

    Returns:
        int: Fitness score of the individual.
    """
    fitness = 0

    for i in range(GRID_SIZE):
        unique_digit_column = np.unique(individual[:, i])
        unique_digit_line = np.unique(individual[i])
        fitness += len(unique_digit_line)
        fitness += len(unique_digit_column)
        
    return fitness

In [ ]:
def selection(population, fitness_scores):
    """
    Selects 50% of the population based on their fitness scores using roulette wheel selection.

    Roulette wheel selection is a widely used method in genetic algorithms for selecting individuals from a population based on their fitness scores. The selection process is analogous to a roulette wheel, where each individual's probability of selection is proportional to their fitness score. Higher fitness scores correspond to larger areas on the roulette wheel, increasing the likelihood of selection.

    Args:
        population (list): List of grid in the current population.
        fitness_scores (list): List of fitness scores corresponding to the individuals/population.

    Returns:
        list: Selected grids based on the selection probabilities.
    """
    nb_individual_to_select = len(population) // 2  

    total_fitness = sum(fitness_scores)
    selection_probabilities = [fit / total_fitness for fit in fitness_scores]

    selected_indices = np.random.choice(np.arange(len(population)), size=nb_individual_to_select, p=selection_probabilities)
    selected_individuals = [population[i] for i in selected_indices]

    return selected_individuals

In [ ]:
def crossover(grid1, grid2):
    """
    Performs crossover between two Sudoku grids to create two descendants.

    Args:
        parent1 (numpy.ndarray): First grid.
        parent2 (numpy.ndarray): Second grid.

    Returns:
        tuple: Two descendants resulting from the crossover.
    """
    descendant1 = np.copy(grid1)
    descendant2 = np.copy(grid2)

    for i in range(GRID_SIZE):
        if i % 2 != 0:
            descendant1[i] = grid2[i]
            descendant2[i] = grid1[i]

    return descendant1, descendant2

In [ ]:
def calculate_mutation_rate(fitness_score):
    """
    Calculates the mutation rate based on the fitness score.

    Args:
        fitness_score (int): Fitness score of a grid (higher score indicates a better solution).

    Returns:
        float: Calculated mutation rate.
    """
    return 1.0 - fitness_score / FITNESS_GOAL

In [ ]:
def mutate(grid):
    """
    Mutates a Sudoku grid by randomly modifying some cells.

    Args:
        grid (numpy.ndarray): Sudoku grid to be mutated.

    Returns:
        numpy.ndarray: Mutated Sudoku grid.
    """
    mutated_grid = np.copy(grid)

    fitness_score = compute_fitness(mutated_grid)
    mutation_rate = calculate_mutation_rate(fitness_score)

    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if np.random.random() < mutation_rate:
                other_possible_values = [num for num in range(1, GRID_SIZE + 1) if num not in grid[i]]
                if len(other_possible_values) != 0:
                    new_value = np.random.choice(other_possible_values)
                else:
                    new_value = np.random.choice(GRID_SIZE)
                mutated_grid[i, j] = new_value

    return mutated_grid

In [ ]:
def generate_new_generation(current_population):
    """
    Generates a new generation of individuals based on the provided functions.

    Args:
        current_population (list of numpy.ndarray): Current population of individuals.

    Returns:
        tuple: A tuple containing a flag indicating whether more generations are needed and the new generation of individuals.
            - bool: True if more generations are needed, False otherwise.
            - list of numpy.ndarray: New generation of individuals.
    """
    fitness_scores = [compute_fitness(grid) for grid in current_population]

    parents = selection(current_population, fitness_scores)

    num_children = len(current_population) - len(parents)

    print(len(parents), len(current_population))
    children = []
    for i in range(num_children // 2):
        idx_parent1, idx_parent2 = np.random.choice(len(parents), size=2, replace=False)
        parent1 = parents[idx_parent1]
        parent2 = parents[idx_parent2]
        child1, child2 = crossover(parent1, parent2)
        children.append(child1)
        children.append(child2)

    mutated_children = [mutate(child) for child in children]

    new_generation = parents + mutated_children

    need_more_generation = True
    if FITNESS_GOAL in fitness_scores :
        need_more_generation = False

    print('Max Fitness', max(fitness_scores))
    return need_more_generation, new_generation

In [ ]:
def main():
    population = generate_population()

    need_more_generation = True
    i = 0
    while need_more_generation:
        print(f'Gen n°{i}')
        need_more_generation, population = generate_new_generation(population)
        i+=1

In [ ]:
if __name__ == "__main__":
    main()